In [83]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
import re
#this is a python magic function to automatically show matplotlib plots
%matplotlib inline  
pd.set_option('display.max_columns', None)

In [60]:
mem = pd.read_csv('../data/NSS_GreatCare_Members.csv')
adm = pd.read_csv('../data/NSS_Hospital_Inpatient_Admissions.csv')
pcp = pd.read_csv('../data/NSS_PrimaryCare_Office_Visits.csv', low_memory = False)
pcp.head(5)

,Encounter ID,Patient ID,Service Provider ID,Attending Provider ID,Provider Group Provider ID,Facility Provider ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI,Encounter Date,Is Qualifying Preventive Primary Care Service,Is Qualifying Primary Care Service Provider,Is Qualifying Primary Care Attending,Is Qualifying Primary Care Provider Group,Qualifying Preventive Primary Care HCPCS Code
0,COV|EE|DS1|10076|20110225|DS1|NPI|1487856779,DS1|10076,DS1|NPI|1487856779,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.487857e+09,1.043427e+09,2011-02-18,0,1,0,0,NaN
1,COV|EE|DS1|10088|20140222|DS1|NPI|1528030996,DS1|10088,DS1|NPI|1528030996,DS1|NPI|1528030996,DS1|NPI|1043427248,DS1|NPI|1902803315,DS1|1,DS1|5047W,1.528031e+09,1.902803e+09,1.528031e+09,1.043427e+09,2014-02-22,0,1,1,1,NaN
2,COV|EE|DS1|10165|20140907|DS1|NPI|1477579365,DS1|10165,DS1|NPI|1477579365,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.477579e+09,1.043427e+09,2014-08-31,0,1,0,0,NaN
3,COV|EE|DS1|10240|20130624|DS1|NPI|1821178088,DS1|10240,DS1|NPI|1821178088,DS1|NPI|1821178088,DS1|NPI|1407058787,DS1|NPI|1104842020,DS1|1,DS1|5047W,1.821178e+09,1.104842e+09,1.821178e+09,1.407059e+09,2013-06-17,0,1,1,1,NaN
4,COV|EE|DS1|10293|20160904|DS1|NPI|1609004191,DS1|10293,DS1|NPI|1609004191,NaN,DS1|NPI|1184722779,NaN,DS1|1,DS1|5047X,NaN,NaN,1.609004e+09,1.184723e+09,2016-09-04,0,1,0,0,NaN


two[['First','second','third','fourth','fifth','sixth','Last']] = two['Encounter ID'].apply(
   lambda x: pd.Series(str(x).split("|")))
two.head()

In [64]:
#set datetime
adm['Encounter Start Date'] = pd.to_datetime(adm['Encounter Start Date'], format='%Y-%m-%d')
adm['Encounter End Date'] = pd.to_datetime(adm['Encounter End Date'], format='%Y-%m-%d')
pcp['Encounter Date'] = pd.to_datetime(pcp['Encounter Date'], format='%Y-%m-%d')

# MVP 1 
What is our 30-day all cause readmission rate across hospitals our members are admitted to?

In [125]:
#make df(patient, start date, end date)
one = adm[['Patient ID','Encounter Start Date','Encounter End Date']]
#order by patient then start date
one = one.sort_values(['Patient ID',"Encounter Start Date", "Encounter End Date"], ascending = (False, True, True))
#shift previous end date to next row and math
one['Previous End Date'] = (one.groupby('Patient ID')['Encounter End Date'].shift(1))
one['Days Since Last Visit'] = one['Encounter Start Date'] - one['Previous End Date']

ad_count = len(one)
ad_count

6213

Different Direction
#if difference between Encounter End Date and Encounter Start Date 
one['diff'] = one.groupby('Patient ID')['Encounter Start Date'].diff() / np.timedelta64(1, 'D')
one['diff'] = one['diff'].fillna(0)
one['diff_end'] = one.groupby('Patient ID')['Encounter End Date'].diff() / np.timedelta64(1, 'D')
one['diff_end'] = one['diff_end'].fillna(0)
#length of stay - not grouped by
one['length'] = one['Encounter End Date']-one['Encounter Start Date'] 
one['length'] = one['length'].fillna(0)

In [132]:
#filter for readmission and within 30 days
one_top = one[['Patient ID', 'Previous End Date', 'Encounter Start Date', 'Encounter End Date', 'Days Since Last Visit']]
one_top = one_top[one_top['Days Since Last Visit'].notnull()]
one_top = one_top[one_top['Days Since Last Visit'] <= '30 days']
one_top = one_top[one_top['Days Since Last Visit'] >= '0 days']
one_top
read_count = len(one_top)
read_count


In [133]:
#find the readmission rate
read_rate = round(read_count/ad_count *100,2)
read_rate

11.31

In [134]:
#find the unique readmission rate 
read_rate_unique = round(one_top['Patient ID'].nunique() / one['Patient ID'].nunique()*100,2)
read_rate_unique

9.47

# MVP 2
What percent of patients have a PCP visit within a week of discharge?

In [123]:
#make df and merge
ad_cut = adm[['Patient ID','Encounter End Date']]
pcp_cut = pcp[['Patient ID','Encounter Date']]
two = ad_cut.merge(pcp_cut, left_on ='Patient ID', right_on = 'Patient ID',how='left', indicator = False)
#sub encounter date from end dates
two['Days Since Last Visit'] = two['Encounter Date'] - two['Encounter End Date']
two_top = two[two['Days Since Last Visit'] <= '7 days']
two_top = two_top[two_top['Days Since Last Visit'] >= '0 days']
#count
count_two_top = len(two_top)
two_top

,Patient ID,Encounter End Date,Encounter Date,Days Since Last Visit
17,DS1|10043,2014-01-12,2014-01-12,0 days
28,DS1|10073,2012-06-21,2012-06-27,6 days
50,DS1|10101,2012-12-09,2012-12-12,3 days
231,DS1|10303,2012-06-24,2012-06-30,6 days
238,DS1|10303,2012-06-24,2012-06-28,4 days
...,...,...,...,...
44051,DS1|9945,2013-02-11,2013-02-14,3 days
44067,DS1|9978,2015-09-05,2015-09-11,6 days
44071,DS1|9979,2015-02-11,2015-02-15,4 days
44074,DS1|9981,2014-08-10,2014-08-11,1 days


In [124]:
#find the pcp visit within 7 days rate
pcp_visit_rate = round(count_two_top/ad_count *100,2)
pcp_visit_rate

38.23

# Question 3
Do either vary by individual or group PCP's?